# Introduction

I'm exploring the data available in the New York Philharmonic’s [Performance History archive](https://github.com/nyphilarchive/PerformanceHistory). We’ll begin by parsing the XML file into a flat structure providing information about the composers and works performed in each season.

In [15]:
from datetime import datetime
import xml.etree.ElementTree as ET
import pandas as pd

## Works

To start with, let's flatten the XML so that each row of the table corresponds to a single work performed at a program with the columns:

- Season (index)
- Program ID (index)
- Work ID (index)
- Movement ID (index)
- Composer
- Title

In [3]:
def get_works(filename):
    root = ET.parse(filename)
    
    for program in root.findall(".//program"):
        program_id = program.find("programID").text
        season = program.find("season").text
    
        for work in program.findall(".//work"):
            try:
                work_id, mvmt_id = work.attrib["ID"].split("*")
            except ValueError:
                work_Id, mvmt_id = work.attrib["ID"], None
            
            yield {
                "program": program_id,
                "work": work_id,
                "movement": mvmt_id,
                "season": season,
                "composer": getattr(work.find("composerName"), "text", None),
                "title": getattr(work.find("workTitle"), "text", None)
            }

In [4]:
df = (pd.DataFrame(list(get_works("../data/PerformanceHistory/Programs/xml/complete.xml")))
      .set_index(["season", "program", "work", "movement"]))
df.head()

composer  \
season  program work  movement                            
1842-43 3853    52446           Beethoven,  Ludwig  van   
                8834  4         Weber,  Carl  Maria Von   
                3642                    Hummel,  Johann   
                0                                  None   
                8834  3         Weber,  Carl  Maria Von   

                                                           title  
season  program work  movement                                    
1842-43 3853    52446           SYMPHONY NO. 5 IN C MINOR, OP.67  
                8834  4                                   OBERON  
                3642             QUINTET, PIANO, D MINOR, OP. 74  
                0                                           None  
                8834  3                                   OBERON

Total number of programmed works:

In [5]:
len(df)

83319

In [6]:
print("Intermissions:", len(df.xs("0", level="work")))

Intermissions: 11333


Drop intermissions (work ID 0).

In [7]:
df.drop(index="0", level="work", inplace=True)
len(df)

71986

Total number of programs:

In [13]:
df.index.get_level_values("program").nunique()

13947

Total number of seasons:

In [14]:
df.index.get_level_values("season").nunique()

177

In [ ]:
df.to_csv("../data/works.csv", encoding="utf-8")

## Program Dates

In [16]:
def get_programs(filename, fmt="%Y-%m-%dT%H:%M:%SZ"):
    root = ET.parse(filename)
    
    for program in root.findall(".//program"):
        program_id = program.find("programID").text
        season = program.find("season").text
        date = datetime.strptime(program.find("concertInfo/Date").text, fmt)
        
        yield {
            "season": season,
            "date": date,
            "program": program_id
        }

In [17]:
df = pd.DataFrame(
    list(get_programs("../data/PerformanceHistory/Programs/xml/complete.xml"))
).set_index("program")

In [18]:
df.head()

,date,season
program,,
3853,1842-12-07 05:00:00,1842-43
5178,1843-02-18 05:00:00,1842-43
10785,1843-04-07 05:00:00,1842-43
5887,1843-04-22 05:00:00,1842-43
305,1843-11-18 05:00:00,1843-44


In [19]:
df.to_csv("../data/program_dates.csv")